### Import the software libraries

# Outline of steps
1. Search the parent directory for all localization files and a build a list of such files
2. Define the drift correction processor
3. Open a file from the list
4. Perform the drift correction on the file
  * If the correction was not good, go back to step 2
4. Save the results
5. Repeat from step 1

Tutorial: https://github.com/kmdouglass/bstore/blob/master/examples/Fiducial-based%20Drift%20Correction.ipynb

In [1]:
%pylab
import bstore.processors as proc
import pandas as pd
from pathlib import Path
import pprint
pp = pprint.PrettyPrinter(indent = 4)

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Search the parent directory and make a list of localization files
We will start by searching a parent directory and its subdirectories for all the localization files, i.e. those files that end in `locResults.dat`.

In [120]:
parentDirectory   = Path('E:/to_analyze/2017-12-08_humanCent_Cep152_Sas6/locResults/')
localizationFiles = parentDirectory.glob('**/Sas6_A647_*Localizations.csv')
locResultFiles    = sorted(localizationFiles)

# How many files are there? 
print(len(locResultFiles))

24


In [121]:
pp.pprint(list(zip(locResultFiles, range(len(locResultFiles)))))

[   (   WindowsPath('E:/to_analyze/2017-12-08_humanCent_Cep152_Sas6/locResults/Sas6_A647_10_1/Sas6_A647_10_1_MMStack_1_Localizations.csv'),
        0),
    (   WindowsPath('E:/to_analyze/2017-12-08_humanCent_Cep152_Sas6/locResults/Sas6_A647_11_1/Sas6_A647_11_1_MMStack_1_Localizations.csv'),
        1),
    (   WindowsPath('E:/to_analyze/2017-12-08_humanCent_Cep152_Sas6/locResults/Sas6_A647_12_1/Sas6_A647_12_1_MMStack_1_Localizations.csv'),
        2),
    (   WindowsPath('E:/to_analyze/2017-12-08_humanCent_Cep152_Sas6/locResults/Sas6_A647_13_1/Sas6_A647_13_1_MMStack_1_Localizations.csv'),
        3),
    (   WindowsPath('E:/to_analyze/2017-12-08_humanCent_Cep152_Sas6/locResults/Sas6_A647_14_1/Sas6_A647_14_1_MMStack_1_Localizations.csv'),
        4),
    (   WindowsPath('E:/to_analyze/2017-12-08_humanCent_Cep152_Sas6/locResults/Sas6_A647_15_1/Sas6_A647_15_1_MMStack_1_Localizations.csv'),
        5),
    (   WindowsPath('E:/to_analyze/2017-12-08_humanCent_Cep152_Sas6/locResults/Sas6_A647

# Define the drift correction processor
This part is the same as in Tutorial 1. At the end of processing for each file, return to this code block and start running the blocks again.

In [221]:
plt.close('all')
# Set the input and output files
# Up to and including are 26 finished
currentFile = locResultFiles[23] # Increment this when done dedrifting a dataset
outputFile  = currentFile.parent / Path(currentFile.stem + '_DC' + currentFile.suffix)

# Create the FiducialDriftCorrect processor and set its properties.
dc = proc.FiducialDriftCorrect(coordCols = ['x [nm]', 'y [nm]'])
dc.driftComputer.smoothingWindowSize = 800
dc.driftComputer.smoothingFilterSize = 200

clean = proc.CleanUp()

# Open a file and clean it up
with open(str(currentFile), 'r') as file:
    df = pd.read_csv(file, sep = ',')
    
# Clean up the data
df = clean(df)

print(str(currentFile))
# df.describe()

E:\to_analyze\2017-12-08_humanCent_Cep152_Sas6\locResults\Sas6_A647_9_1\Sas6_A647_9_1_MMStack_1_Localizations.csv


In [222]:
corrdf = dc(df)
dc.driftComputer.plotFiducials()

Performing spline fits...


In [228]:
plt.close('all')
# Skip this cell if you don't need to adjust the fits
dc.interactiveSearch = False

# Modify these if needed to adjust the fits
dc.driftComputer.useTrajectories     = [0] # Set to [] if you want to use all fiducials
dc.driftComputer.zeroFrame           = 1000# Set to 1000 by default
dc.driftComputer.smoothingWindowSize = 1800  # Set to 800 by default
dc.driftComputer.smoothingFilterSize = 300   # Set to 300 by default
dc.driftComputer.maxRadius = 500

# processed_df = dc(df)
corrdf = dc(df)
dc.driftComputer.plotFiducials()

Performing spline fits...


In [ ]:
# df.describe()
# processed_df.head()
corrdf.head()

In [ ]:
# Run this only if no good fiducials are found.
# plt.close('all')
# dc.driftComputer.fiducialLocs = None

In [229]:
saveTrajectories = True
# If no fiducials were found, append '_DCX' to the filename;
# Otherwise, append '_DC'
if dc.driftComputer.fiducialLocs is None:
    with open(str(currentFile), 'r') as file:
        df = pd.read_csv(file)

    corrdf = clean(df)
    outputFile = currentFile.parent / Path(currentFile.stem + '_DCX' + currentFile.suffix)
    saveTrajectories = False
    
print(str(outputFile))

# Save the data
with open(str(outputFile), 'w') as file:
    corrdf.to_csv(file, index=False)
    
if saveTrajectories:
    # Save the average trajectory
    #outputTrajectoryFile = currentFile.parent / Path(currentFile.stem + '_AverageFiducialTrajectory' + currentFile.suffix)
    outputTrajectoryFile = currentFile.parent / Path(currentFile.stem + '_Avg' + currentFile.suffix)
    with open(str(outputTrajectoryFile), 'w') as file:
        dc.driftTrajectory.to_csv(file, index = True)
    print(str(outputTrajectoryFile))
    
    # Save the individual trajectories
    #outputTrajectoryFile = currentFile.parent / Path(currentFile.stem + '_FiducialTrajectories' + currentFile.suffix)
    outputTrajectoryFile = currentFile.parent / Path(currentFile.stem + '_Fid' + currentFile.suffix)
    #with open(str(outputTrajectoryFile), 'w') as file:
    #   dc.driftComputer.fiducialLocs.loc[(slice(None), dc.driftComputer.useTrajectories), :].to_csv(file, index = True)
    #print(str(outputTrajectoryFile))
    
    with open(str(outputTrajectoryFile), 'w') as file:
            if dc.driftComputer.useTrajectories: # Important because the.loc will return None if all trajectories are used 
                dc.driftComputer.fiducialLocs.loc[(slice(None),dc.driftComputer.useTrajectories), :].to_csv(file, index = True)
            else:
                dc.driftComputer.fiducialLocs.to_csv(file, index = True)
    print(str(outputTrajectoryFile))
    
    plt.close('all')

E:\to_analyze\2017-12-08_humanCent_Cep152_Sas6\locResults\Sas6_A647_9_1\Sas6_A647_9_1_MMStack_1_Localizations_DC.csv
E:\to_analyze\2017-12-08_humanCent_Cep152_Sas6\locResults\Sas6_A647_9_1\Sas6_A647_9_1_MMStack_1_Localizations_Avg.csv
E:\to_analyze\2017-12-08_humanCent_Cep152_Sas6\locResults\Sas6_A647_9_1\Sas6_A647_9_1_MMStack_1_Localizations_Fid.csv


Now you can cycle back up to the beginning and increase the index locResultFiles array by one and repeat the process in this notebook.